# Train Music Transformer
Since Fri. Feb. 25th, 2022

Set up training in colab


## Setup


### Ipython



In [6]:
%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Colab



In [7]:
import os
import sys

import torch


if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    ! pip3 install sty icecream music21 transformers datasets==1.18.3  # for `set_progress_bar_enabled`

    path = os.path.join('drive', 'My Drive', 'Research', 'Music with NLP', 'Symbolic-Music-Generation')
    sys.path.append(path)
    ! ls "{path}"


    import time, os
    os.environ['TZ'] = 'US/Eastern'
    time.tzset()

    if torch.cuda.is_available():
        %env PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128

if torch.cuda.is_available():
    ! nvidia-smi


from icecream import ic

from musicnlp.util import *

ic(PATH_BASE, DIR_PROJ, PKG_NM)



ic| PATH_BASE: '/Users/stefanh/Documents/UMich/Research/Music with NLP'
    DIR_PROJ: 'Symbolic-Music-Generation'
    PKG_NM: 'musicnlp'


('/Users/stefanh/Documents/UMich/Research/Music with NLP',
 'Symbolic-Music-Generation',
 'musicnlp')

### code

In [8]:
from musicnlp.util import *
from musicnlp.models import train


seed = config('random-seed')



## Prep for training


In [9]:
dnm_909 = 'musicnlp music extraction, dnm=POP909, n=909, meta={mode=melody, prec=5, th=1}, 2022-04-10_12-51-01'
dnm_lmd = 'musicnlp music extraction, dnm=LMD-cleaned-subset, ' \
          'n=10269, meta={mode=melody, prec=5, th=1}, 2022-04-10_12-52-41'
dnms = [dnm_909, dnm_lmd]

md_nm = 'reformer'
md_sz = 'tiny'
# md_sz = 'base'


if md_nm != 'reformer':
    import transformers
    transformers.set_seed(seed)


if md_sz in ['debug', 'tiny']:  # debugging
    n = 8
    train_args = dict(
        per_device_train_batch_size=4,
        # save_strategy='no',
        save_strategy='epoch',
        num_train_epochs=64,
    )
    my_train_args = dict(
        save_epochs=16
    )
else:
    n = None
    train_args = dict(num_train_epochs=16)
    my_train_args = dict(
        fp16=torch.cuda.is_available(),
        logging_strategy='epoch',
        save_epochs=4
    )

mdl, tokenizer, trainer = train.get_all_setup(
    model_name=md_nm, model_size=md_sz, dataset_names=dnms, n_sample=n, dataset_seed=seed,
    train_args=train_args, my_train_args=my_train_args
)



Loading cached processed dataset at /Users/stefanh/Documents/UMich/Research/Music with NLP/datasets/MNLP-Combined/processed/musicnlp music extraction, dnm=POP909, n=909, meta={mode=melody, prec=5, th=1}, 2022-04-10_12-51-01/train/cache-0ade476f1b4e9e0c.arrow
Loading cached processed dataset at /Users/stefanh/Documents/UMich/Research/Music with NLP/datasets/MNLP-Combined/processed/musicnlp music extraction, dnm=POP909, n=909, meta={mode=melody, prec=5, th=1}, 2022-04-10_12-51-01/train/cache-5d57d94a1bfb3813.arrow
Loading cached processed dataset at /Users/stefanh/Documents/UMich/Research/Music with NLP/datasets/MNLP-Combined/processed/musicnlp music extraction, dnm=POP909, n=909, meta={mode=melody, prec=5, th=1}, 2022-04-10_12-51-01/train/cache-ae59fc2f8b1b097f.arrow
Loading cached processed dataset at /Users/stefanh/Documents/UMich/Research/Music with NLP/datasets/MNLP-Combined/processed/musicnlp music extraction, dnm=POP909, n=909, meta={mode=melody, prec=5, th=1}, 2022-04-10_12-51-01

## Train


In [10]:
trainer.train()
trainer.save_model(os.path.join(trainer.args.output_dir, 'trained'))


2022-04-10 16:48:26|[ReformerModelWithLMHead Training]::on_train_begin::train.py:180, INFO - Training started with model {model name: ReformerModelWithLMHead, max length: 1024, axial_pos_shape: (32, 32), n_layer: 6, hidden_size: 256, ff_size: 1024, attention_shape: 8x32, parameter_count: 4.6M}, {
    "hash_seed": null,
    "vocab_size": 390,
    "attention_head_size": 32,
    "hidden_size": 256,
    "num_attention_heads": 8,
    "num_hashes": 1,
    "num_hidden_layers": 6,
    "num_buckets": null,
    "lsh_attn_chunk_length": 64,
    "local_attn_chunk_length": 64,
    "lsh_num_chunks_after": 0,
    "lsh_num_chunks_before": 1,
    "local_num_chunks_after": 0,
    "local_num_chunks_before": 1,
    "hidden_act": "relu",
    "feed_forward_size": 1024,
    "hidden_dropout_prob": 0.05,
    "lsh_attention_probs_dropout_prob": 0.0,
    "local_attention_probs_dropout_prob": 0.05,
    "max_position_embeddings": 1024,
    "initializer_range": 0.02,
    "layer_norm_eps": 1e-12,
    "axial_pos_embd

config.num_buckets is not set. Setting config.num_buckets to 32...
config.num_buckets is not set. Setting config.num_buckets to 32...
config.num_buckets is not set. Setting config.num_buckets to 32...


2022-04-10 16:48:29|[ReformerModelWithLMHead Training]::_log::train.py:249, INFO - {step:   1/128, epoch:  0.500/64, train/learning_rate: 2.310e-05, train/loss: 6.0604, train/ntp_acc:  0.17, train/ikr:  9.57}
2022-04-10 16:48:33|[ReformerModelWithLMHead Training]::_log::train.py:249, INFO - {step:   2/128, epoch:  1.000/64, train/learning_rate: 4.620e-05, train/loss: 6.0626, train/ntp_acc:  0.12, train/ikr: 12.46}
2022-04-10 16:48:35|[ReformerModelWithLMHead Training]::_log::train.py:249, INFO - {step:   2/128, epoch:  1/64, eval/loss: 6.0068, eval/ntp_acc:  0.21, eval/ikr:  9.44}
2022-04-10 16:48:38|[ReformerModelWithLMHead Training]::_log::train.py:249, INFO - {step:   3/128, epoch:  1.500/64, train/learning_rate: 6.920e-05, train/loss:  6.01, train/ntp_acc:  0.39, train/ikr: 12.12}


KeyboardInterrupt: 

Check log and tensorboard files written



In [ ]:
ic(trainer.args.output_dir)
os.listdir(trainer.args.output_dir)

